In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import mxnet as mx

from joblib import Parallel, delayed
import time

In [2]:
# labels =  ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']

labels = ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL', 
                     '6N', 'Amb', 'R', 'Tz', 'Sol', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

label_dict = dict([(l,i) for i, l in enumerate(labels)])

# label_dict = dict([(l,i) for i, l in enumerate(labels)] + \
#                   zip(other_labels, range(len(labels), len(labels)+len(other_labels))))

In [3]:
patches_rootdir = '/home/yuncong/CSHL_data_patches/'
model_dir = '/home/yuncong/mxnet_models/'

In [4]:
batch_size = 32

In [ ]:
# model_name='Inception_BN'
# model_iteration = 39
# init_model = mx.model.FeedForward.load(os.path.join(model_dir, model_name), 
#                                      model_iteration, 
#                                      ctx=mx.gpu(),
#                                     numpy_batch_size=batch_size)

# arg_params = init_model.arg_params.copy()
# arg_params.pop('fc_bias');
# arg_params.pop('fc_weight');

In [5]:
model_name='inception-stage1'
model_iteration = 10
init_model = mx.model.FeedForward.load(os.path.join(model_dir, model_name), 
                                     model_iteration, 
                                     ctx=mx.gpu(),
                                    numpy_batch_size=batch_size)


arg_params = init_model.arg_params.copy()
arg_params.pop('fullc_bias');
arg_params.pop('fullc_weight');

In [6]:
n_class = len(labels)
internals = init_model.symbol.get_internals()
output = internals['flatten_output']
fc = mx.symbol.FullyConnected(data=output, name='fullc', num_hidden=n_class)
softmax = mx.symbol.SoftmaxOutput(data=fc, name='softmax')

In [7]:
n_epoch = 10
l_rate = 0.001
checkpoint_prefix = os.path.join(model_dir, 'experiment0227')

In [8]:
opt = mx.optimizer.SGD(learning_rate=l_rate)

net = mx.model.FeedForward(ctx=mx.gpu(), 
#                            symbol=init_model.symbol, 
                           symbol=softmax, 
                           num_epoch=n_epoch, optimizer=opt,
                           arg_params = arg_params, 
                           aux_params = init_model.aux_params,
                           allow_extra_params = True,
                           numpy_batch_size = batch_size)

In [9]:
mean_img = mx.nd.load(os.path.join(model_dir, 'mean_224.nd'))['mean_img'].asnumpy()

In [10]:
stack = 'MD589'
stack_has_annotation = True

if stack_has_annotation:
    table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allLandmarks_allSections.h5'%{'stack':stack})
    indices_allLandmarks_allSections = pd.read_hdf(table_filepath, 'indices_allLandmarks_allSections')
    grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

first_bs_sec, last_bs_sec = section_range_lookup[stack]
first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

In [11]:
patches_allClasses = []
patchLabels_allClasses = []

for sec in range(first_detect_sec, last_detect_sec+1):
# for sec in range(first_detect_sec, first_detect_sec+20):

    if sec not in indices_allLandmarks_allSections.columns:
        continue

    print sec

    ## define grid, generate patches

#     t = time.time()

    dm = DataManager(stack=stack, section=sec)
    dm._load_image(['rgb-jpg'])

    patch_size, stride, w, h = grid_parameters.tolist()
    half_size = patch_size/2

    ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                     indexing='xy')

    sample_locations = np.c_[xs.flat, ys.flat]
    
    
    q = indices_allLandmarks_allSections[sec].dropna()
    if len(q.index) == 0:
        continue
    
    for label_ind, label in enumerate(labels):
        
        print label

        if label == 'BackG':
            w = [q[l] for l in q.index if l.endswith('surround')]
            if len(w) == 0:
                continue
            else:
                indices_roi = np.concatenate(w)
        else:
            if label in q.index:
                indices_roi = q[label]
            else:
                continue
        
        n2 = len(indices_roi)
        print n2, 'samples'
        
        indices_roi = np.random.choice(indices_roi, min(30, n2), replace=False)

        n = len(indices_roi)
        print n, 'used samples'
                
        sample_locations_roi = sample_locations[indices_roi]
        
        patches2 = np.asarray([dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
                              for x, y in sample_locations_roi])
        
        patches = np.rollaxis(patches2, 3, 1)
        patches = patches - mean_img
        
        patches_allClasses.append(patches.copy())
        
        patch_labels = label_ind * np.ones((n, ), np.int)
        patchLabels_allClasses.append(patch_labels)
        
        del patches, patches2, sample_locations_roi

    del sample_locations
    
#     break
#     sys.stderr.write('generating patches: %.2f seconds\n' % (time.time() - t))
    

150
BackG
1662 samples
30 used samples
5N
121 samples
30 used samples
7n
48 samples
30 used samples
7N
12N
Gr
LVe
Pn
SuVe
VLL
83 samples
30 used samples
6N
Amb
R
Tz
Sol
RtTg
LRt
LC
AP
sp5
151
BackG
4185 samples
30 used samples
5N
72 samples
30 used samples
7n
49 samples
30 used samples
7N
12N
Gr
LVe
Pn
SuVe
VLL
116 samples
30 used samples
6N
Amb
R
Tz
Sol
RtTg
LRt
LC
AP
sp5
315 samples
30 used samples
152
BackG
1907 samples
30 used samples
5N
122 samples
30 used samples
7n
68 samples
30 used samples
7N
29 samples
29 used samples
12N
Gr
LVe
Pn
SuVe
VLL
7 samples
7 used samples
6N
Amb
R
Tz
Sol
RtTg
LRt
LC
AP
sp5
153
BackG
4166 samples
30 used samples
5N
231 samples
30 used samples
7n
46 samples
30 used samples
7N
90 samples
30 used samples
12N
Gr
LVe
Pn
SuVe
VLL
20 samples
20 used samples
6N
Amb
R
Tz
Sol
RtTg
LRt
LC
AP
sp5
159 samples
30 used samples
154
BackG
5355 samples
30 used samples
5N
198 samples
30 used samples
7n
65 samples
30 used samples
7N
80 samples
30 used samples
12N
Gr
LVe

In [13]:
patches_allClasses_arr = np.concatenate(patches_allClasses)
patchLabels_allClasses_arr = np.concatenate(patchLabels_allClasses)

In [14]:
len(patchLabels_allClasses_arr)

32704

In [15]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [16]:
batch_size = 32

train_iter = mx.io.NDArrayIter(
    patches_allClasses_arr, 
    patchLabels_allClasses_arr,
    batch_size = batch_size,
    shuffle=True
)

t = time.time()

net.fit(train_iter,
        batch_end_callback=mx.callback.Speedometer(batch_size, 30),
        epoch_end_callback=mx.callback.do_checkpoint(checkpoint_prefix))

net.save(checkpoint_prefix)

print time.time() - t

# 100 samples/sec


INFO:root:Start training with [gpu(0)]
INFO:root:Epoch[0] Batch [30]	Speed: 104.43 samples/sec	Train-accuracy=0.491667
INFO:root:Epoch[0] Batch [60]	Speed: 101.31 samples/sec	Train-accuracy=0.590625
INFO:root:Epoch[0] Batch [90]	Speed: 101.14 samples/sec	Train-accuracy=0.636111
INFO:root:Epoch[0] Batch [120]	Speed: 100.70 samples/sec	Train-accuracy=0.667188
INFO:root:Epoch[0] Batch [150]	Speed: 100.18 samples/sec	Train-accuracy=0.692083
INFO:root:Epoch[0] Batch [180]	Speed: 99.92 samples/sec	Train-accuracy=0.714757
INFO:root:Epoch[0] Batch [210]	Speed: 99.90 samples/sec	Train-accuracy=0.728720
INFO:root:Epoch[0] Batch [240]	Speed: 99.76 samples/sec	Train-accuracy=0.740234
INFO:root:Epoch[0] Batch [270]	Speed: 99.65 samples/sec	Train-accuracy=0.750116
INFO:root:Epoch[0] Batch [300]	Speed: 97.81 samples/sec	Train-accuracy=0.754062
INFO:root:Epoch[0] Batch [330]	Speed: 94.47 samples/sec	Train-accuracy=0.759375
INFO:root:Epoch[0] Batch [360]	Speed: 95.98 samples/sec	Train-accuracy=0.766667

KeyboardInterrupt: 

In [ ]:
q = np.where(patchLabels_allClasses == label_dict['7N'])[0]

for p in patches_allClasses[q]:
    plt.imshow(np.rollaxis(p + mean_img, 0, 3).astype(np.uint8));
    plt.show();